In [1]:
import pandas as pd
import re

### Read csv

In [2]:
df = pd.read_csv("qual_data.csv")

### Combine list of unused columns

In [3]:
columns_to_drop = [
    "StartDate",
    "EndDate",
    "Status",
    "IPAddress",
    "RecordedDate",
    "RecipientLastName",
    "RecipientFirstName",
    "RecipientEmail",
    "ExternalReference",
    "LocationLatitude",
    "LocationLongitude",
    "DistributionChannel",
    "UserLanguage",
    "consent",
    "combination",
    "priorExperience",
    "Q111",
    "Q24",
    "Q25",
]

### Drop unused columns

In [4]:
df = df.drop(columns_to_drop, axis=1)

### Drop duplicate ResponseId

In [5]:
df = df.drop(df.iloc[:, [66]], axis=1)

### Create list of columns for renaming

In [6]:
rename_columns = {
    "Q93_1":"t3_1",
    "Q93_2":"t3_2",
    "Q93_3":"t3_3",
    "Q93_4":"t3_4",
    "Q98_1":"t6_1",
    "Q98_2":"t6_2",
    "Q98_3":"t6_3",
    "Q98_4":"t6_4",
    "Q99_1":"t9_1",
    "Q99_2":"t9_2",
    "Q99_3":"t9_3",
    "Q99_4":"t9_4",
    "Q100_1":"t12_1",
    "Q100_2":"t12_2",
    "Q100_3":"t12_3",
    "Q100_4":"t12_4",
    "Q101_1":"t15_1",
    "Q101_2":"t15_2",
    "Q101_3":"t15_3",
    "Q101_4":"t15_4",
    "Q102_1":"t18_1",
    "Q102_2":"t18_2",
    "Q102_3":"t18_3",
    "Q102_4":"t18_4",
    "Q103_1":"t21_1",
    "Q103_2":"t21_2",
    "Q103_3":"t21_3",
    "Q103_4":"t21_4",
    "Q104_1":"t24_1",
    "Q104_2":"t24_2",
    "Q104_3":"t24_3",
    "Q104_4":"t24_4",
    "Q105_1":"t27_1",
    "Q105_2":"t27_2",
    "Q105_3":"t27_3",
    "Q105_4":"t27_4",
    "Q106_1":"t30_1",
    "Q106_2":"t30_2",
    "Q106_3":"t30_3",
    "Q106_4":"t30_4",
    "Q107_1":"t33_1",
    "Q107_2":"t33_2",
    "Q107_3":"t33_3",
    "Q107_4":"t33_4",
    "Q108_1": "t36_1",
    "Q108_2": "t36_2",
    "Q108_3": "t36_3",
    "Q108_4": "t36_4",
    "feature_usefullness_1" : "sidebar_rating",
    "feature_usefullness_2" : "categorization_rating",
    "feature_usefullness_3" : "category_sidebar_rating"
}

### Rename the columns

In [7]:
df = df.rename(rename_columns, axis=1)

### Drop 1st two rows containing redundant data(question test, import id)

In [8]:
df = df.drop(df.index[0:2])

### Combine list of columns to change the data type

In [9]:
correct_dtypes = {
    "t3_1": float,
    "t3_2": float,
    "t3_3": float,
    "t3_4": float,
    "t6_1": float,
    "t6_2": float,
    "t6_3": float,
    "t6_4": float,
    "t9_1": float,
    "t9_2": float,
    "t9_3": float,
    "t9_4": float,
    "t12_1":float,
    "t12_2":float,
    "t12_3":float,
    "t12_4":float,
    "t15_1":float,
    "t15_2":float,
    "t15_3":float,
    "t15_4":float,
    "t18_1":float,
    "t18_2":float,
    "t18_3":float,
    "t18_4":float,
    "t21_1":float,
    "t21_2":float,
    "t21_3":float,
    "t21_4":float,
    "t24_1":float,
    "t24_2":float,
    "t24_3":float,
    "t24_4":float,
    "t27_1":float,
    "t27_2":float,
    "t27_3":float,
    "t27_4":float,
    "t30_1":float,
    "t30_2":float,
    "t30_3":float,
    "t30_4":float,
    "t33_1":float,
    "t33_2":float,
    "t33_3":float,
    "t33_4":float,
    "t36_1":float,
    "t36_2":float,
    "t36_3":float,
    "t36_4":float,
    "sidebar_rating":float,
    "categorization_rating":float,
    "category_sidebar_rating":float
}

### Change the data type

In [10]:
df = df.astype(correct_dtypes)

### Create functions to combine the 4 questions for ease_of_use using System Usability Scale method

In [11]:
def process_t3(row):
    if row.IV1 == "fixed-sidebar":
        return 2.5*( (row.t3_1+4-1) + (7 - row.t3_2 - 4) + (row.t3_3 + 4-1) + (7 - row.t3_4 - 4) )
    else:
        return None
    
def process_t6(row):
    if row.IV1 == "fixed-sidebar":
        return 2.5*( (row.t6_1 + 4 - 1) + (7 - row.t6_2 - 4) + (row.t6_3 + 4 - 1) + (7 - row.t6_4 - 4) )
    else:
        return None

def process_t9(row):
    if row.IV1 == "fixed-sidebar":
        return 2.5*( (row.t9_1+4-1) + (7 - row.t9_2 - 4) + (row.t9_3 + 4 -1) + (7 - row.t9_4 - 4) )
    else:
        return None
        
def process_t12(row):
    print(row)
    if row.IV1 == "fixed-sidebar":
        return 2.5*( (row.t12_1 + 4 -1) + (7 - row.t12_2 - 4) + (row.t12_3 + 4 -1) + (7 - row.t12_4 - 4) )
    else:
        return None
    
def process_t15(row):
    if row.IV1 == "fixed-sidebar":
        return 2.5*( (row.t15_1 + 4 -1) + (7 - row.t15_2 - 4) + (row.t15_3 + 4 -1) + (7 - row.t15_4 - 4) )
    else:
        return None
    
def process_t18(row):
    print(row)
    if row.IV1 == "fixed-sidebar":
        return 2.5*( (row.t18_1 + 4 -1) + (7 - row.t18_2 - 4) + (row.t18_3 + 4 -1) + (7 - row.t18_4 - 4) )
    else:
        return None

def process_t21(row):
    if row.IV1 == "floating-sidebar":
        return 2.5*( (row.t21_1 + 4 -1) + (7 - row.t21_2 - 4) + (row.t21_3 + 4 -1) + (7 - row.t21_4 - 4) )
    else:
        return None
    
def process_t24(row):
    if row.IV1 == "floating-sidebar":
        return 2.5*( (row.t24_1 + 4 -1) + (7 - row.t24_2 - 4) + (row.t24_3 + 4 -1) + (7 - row.t24_4 - 4) )
    else:
        return None

def process_t27(row):
    if row.IV1 == "floating-sidebar":
        return 2.5*( (row.t27_1 + 4 -1) + (7 - row.t27_2 - 4) + (row.t27_3 + 4 -1) + (7 - row.t27_4 - 4) )
    else:
        return None
        
def process_t30(row):
    if row.IV1 == "floating-sidebar":
        return 2.5*( (row.t30_1 + 4 -1) + (7 - row.t30_2 - 4) + (row.t30_3 + 4 -1) + (7 - row.t30_4 - 4) )
    else:
        return None
    
def process_t33(row):
    if row.IV1 == "floating-sidebar":
        return 2.5*( (row.t33_1 + 4 -1) + (7 - row.t33_2 - 4) + (row.t33_3 + 4 -1) + (7 - row.t33_4 - 4) )
    else:
        return None
    
def process_t36(row):
    if row.IV1 == "floating-sidebar":
        return 2.5*( (row.t36_1 + 4 -1) + (7 - row.t36_2 - 4) + (row.t36_3 + 4 -1) + (7 - row.t36_4 - 4) )
    else:
        return None

### Apply the previous methods to trial question and combine them together into 1 metric
### Questions were asked after every 3 trials, so broadcast the same answers for previous 2 trials

In [12]:
df["t3"] = df.apply(lambda row: process_t3(row), axis=1)
df["t1"] = df.apply(lambda row:row.t3, axis=1)
df["t2"] = df.apply(lambda row:row.t3, axis=1)

df["t6"] = df.apply(lambda row: process_t6(row), axis=1)
df["t4"] = df.apply(lambda row:row.t6, axis=1)
df["t5"] = df.apply(lambda row:row.t6, axis=1)

df["t9"] = df.apply(lambda row: process_t9(row), axis=1)
df["t7"] = df.apply(lambda row:row.t9, axis=1)
df["t8"] = df.apply(lambda row:row.t9, axis=1)

df["t12"] = df.apply(lambda row: process_t12(row), axis=1)
df["t10"] = df.apply(lambda row:row.t12, axis=1)
df["t11"] = df.apply(lambda row:row.t12, axis=1)

df["t15"] = df.apply(lambda row: process_t15(row), axis=1)
df["t13"] = df.apply(lambda row:row.t15, axis=1)
df["t14"] = df.apply(lambda row:row.t15, axis=1)

df["t18"] = df.apply(lambda row: process_t18(row), axis=1)
df["t16"] = df.apply(lambda row:row.t18, axis=1)
df["t17"] = df.apply(lambda row:row.t18, axis=1)

df["t21"] = df.apply(lambda row: process_t21(row), axis=1)
df["t19"] = df.apply(lambda row:row.t21, axis=1)
df["t20"] = df.apply(lambda row:row.t21, axis=1)

df["t24"] = df.apply(lambda row: process_t24(row), axis=1)
df["t22"] = df.apply(lambda row:row.t24, axis=1)
df["t23"] = df.apply(lambda row:row.t24, axis=1)

df["t27"] = df.apply(lambda row: process_t27(row), axis=1)
df["t25"] = df.apply(lambda row:row.t27, axis=1)
df["t26"] = df.apply(lambda row:row.t27, axis=1)

df["t30"] = df.apply(lambda row: process_t30(row), axis=1)
df["t28"] = df.apply(lambda row:row.t30, axis=1)
df["t29"] = df.apply(lambda row:row.t30, axis=1)

df["t33"] = df.apply(lambda row: process_t33(row), axis=1)
df["t31"] = df.apply(lambda row:row.t33, axis=1)
df["t32"] = df.apply(lambda row:row.t33, axis=1)

df["t36"] = df.apply(lambda row: process_t36(row), axis=1)
df["t34"] = df.apply(lambda row:row.t36, axis=1)
df["t35"] = df.apply(lambda row:row.t36, axis=1)

Progress                               100
Duration (in seconds)                  565
Finished                                 1
ResponseId               R_1jg3libD6KGk9H6
pri_exp                                  1
                               ...        
t4                                     NaN
t5                                     NaN
t9                                     NaN
t7                                     NaN
t8                                     NaN
Name: 2, Length: 78, dtype: object
Progress                               100
Duration (in seconds)                  100
Finished                                 1
ResponseId               R_28J2ielzrCfvy71
pri_exp                                  1
                               ...        
t4                                    27.5
t5                                    27.5
t9                                    35.0
t7                                    35.0
t8                                    35.0
Name: 3, Length: 78

### Drop the individual columns for each question since their data has been combined into 1 single column

In [13]:
drop_t_columns = {
"t3_1",
"t3_2",
"t3_3",
"t3_4",
"t6_1",
"t6_2",
"t6_3",
"t6_4",
"t9_1",
"t9_2",
"t9_3",
"t9_4",
"t12_1",
"t12_2",
"t12_3",
"t12_4",
"t15_1",
"t15_2",
"t15_3",
"t15_4",
"t18_1",
"t18_2",
"t18_3",
"t18_4",
"t21_1",
"t21_2",
"t21_3",
"t21_4",
"t24_1",
"t24_2",
"t24_3",
"t24_4",
"t27_1",
"t27_2",
"t27_3",
"t27_4",
"t30_1",
"t30_2",
"t30_3",
"t30_4",
"t33_1",
"t33_2",
"t33_3",
"t33_4",
"t36_1",
"t36_2",
"t36_3",
"t36_4"
}

In [14]:
df = df.drop(drop_t_columns, axis=1)

### Drop some more unused columns

In [15]:
df = df.drop(["platform","Duration (in seconds)"], axis=1)

### Convert from wide to long form

In [17]:
df2 = pd.melt(df, id_vars=["Progress", "Finished","ResponseId","pri_exp","buy_check","browser_check","task_check","age_group","gender","computer_use_confide","sweet_food_affinity","computer_os","sidebar_rating","categorization_rating","category_sidebar_rating","IV1","arrangement","sequence" ,"ecomm_frequence"  ], value_vars = ["t1","t2","t3","t4","t5","t6","t7","t8","t9","t10","t11","t12","t13","t14","t15","t16","t17","t18","t19","t21","t22","t23","t24","t25","t26","t27","t28","t29","t30","t31","t32","t33","t34","t35","t36"], var_name="trial_id",value_name='ease_of_use')

### Remove the rows with NA in ease_of_use column

In [18]:
df3 = df2[df2['ease_of_use'].notna()]

In [19]:
df3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 123 entries, 1 to 310
Data columns (total 21 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Progress                 123 non-null    object 
 1   Finished                 123 non-null    object 
 2   ResponseId               123 non-null    object 
 3   pri_exp                  123 non-null    object 
 4   buy_check                123 non-null    object 
 5   browser_check            123 non-null    object 
 6   task_check               123 non-null    object 
 7   age_group                123 non-null    object 
 8   gender                   123 non-null    object 
 9   computer_use_confide     123 non-null    object 
 10  sweet_food_affinity      123 non-null    object 
 11  computer_os              123 non-null    object 
 12  sidebar_rating           123 non-null    float64
 13  categorization_rating    123 non-null    float64
 14  category_sidebar_rating  1

### Remove some more unused columns

In [20]:
df3 = df3.drop(["Progress","Finished","task_check","browser_check","sequence","arrangement"], axis=1)

### Convert data type of trial_id columns to int

In [21]:
df3["trial_id"] = df3["trial_id"].str.replace(r'\D+', '').astype('int')

<ipython-input-21-7e6a4762c9ee>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df3["trial_id"] = df3["trial_id"].str.replace(r'\D+', '').astype('int')


### Function for getting the level of IV2 and IV3 using the trial_id

In [22]:
def make_iv2(row):
    if any(lower<=row.trial_id<=upper for (lower, upper) in [(1,3), (10,12),(19,21),(28,30)]):
        return "right_sidebar_no_categorization"
    elif any(lower<=row.trial_id<=upper for (lower, upper) in [(8,9), (16,18),(25,27),(34,26)]):
        return "no_right_sidebar_no_categorization"
    else:
        return "right_sidebar_categorization"

def make_iv3(row):
    if any(lower<=row.trial_id<=upper for (lower, upper) in [(1,9), (19,27)]):
        return "6-cupcakes"
    else:
        return "3-cupcakes"        

### Apply the functions previously defined to create columns for IV2 and IV3

In [23]:
df3["IV2"] = df3.apply(lambda row:make_iv2(row),axis=1)
df3["IV3"] = df3.apply(lambda row:make_iv3(row),axis=1)

In [24]:
print(df3[["ResponseId","IV1","IV2","IV3","trial_id","ease_of_use"]].to_string())

            ResponseId               IV1                                 IV2         IV3  trial_id  ease_of_use
1    R_28J2ielzrCfvy71     fixed-sidebar     right_sidebar_no_categorization  6-cupcakes         1         25.0
3    R_1lsl8kCjFiBTyzK     fixed-sidebar     right_sidebar_no_categorization  6-cupcakes         1         35.0
7    R_UfKVwc8Ysy6pN1n     fixed-sidebar     right_sidebar_no_categorization  6-cupcakes         1         32.5
8    R_1cTAX495KgNw4zW     fixed-sidebar     right_sidebar_no_categorization  6-cupcakes         1         30.0
10   R_28J2ielzrCfvy71     fixed-sidebar     right_sidebar_no_categorization  6-cupcakes         2         25.0
12   R_1lsl8kCjFiBTyzK     fixed-sidebar     right_sidebar_no_categorization  6-cupcakes         2         35.0
16   R_UfKVwc8Ysy6pN1n     fixed-sidebar     right_sidebar_no_categorization  6-cupcakes         2         32.5
17   R_1cTAX495KgNw4zW     fixed-sidebar     right_sidebar_no_categorization  6-cupcakes         2      

### Drop some more unused columns

In [28]:
df3 = df3.drop(["pri_exp","buy_check"], axis=1)

### List of final columns

In [31]:
df3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 123 entries, 1 to 310
Data columns (total 15 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   ResponseId               123 non-null    object 
 1   age_group                123 non-null    object 
 2   gender                   123 non-null    object 
 3   computer_use_confide     123 non-null    object 
 4   sweet_food_affinity      123 non-null    object 
 5   computer_os              123 non-null    object 
 6   sidebar_rating           123 non-null    float64
 7   categorization_rating    123 non-null    float64
 8   category_sidebar_rating  123 non-null    float64
 9   IV1                      123 non-null    object 
 10  ecomm_frequence          123 non-null    object 
 11  trial_id                 123 non-null    int32  
 12  ease_of_use              123 non-null    float64
 13  IV2                      123 non-null    object 
 14  IV3                      1

### First 5 rows in cleaned data

In [29]:
df3.head()

,ResponseId,age_group,gender,computer_use_confide,sweet_food_affinity,computer_os,sidebar_rating,categorization_rating,category_sidebar_rating,IV1,ecomm_frequence,trial_id,ease_of_use,IV2,IV3
1,R_28J2ielzrCfvy71,2,1,3,-1,1,7.0,6.0,5.0,fixed-sidebar,2,1,25.0,right_sidebar_no_categorization,6-cupcakes
3,R_1lsl8kCjFiBTyzK,2,1,3,3,1,7.0,7.0,7.0,fixed-sidebar,2,1,35.0,right_sidebar_no_categorization,6-cupcakes
7,R_UfKVwc8Ysy6pN1n,2,2,2,2,1,7.0,3.0,2.0,fixed-sidebar,2,1,32.5,right_sidebar_no_categorization,6-cupcakes
8,R_1cTAX495KgNw4zW,2,1,2,-1,1,6.0,5.0,7.0,fixed-sidebar,3,1,30.0,right_sidebar_no_categorization,6-cupcakes
10,R_28J2ielzrCfvy71,2,1,3,-1,1,7.0,6.0,5.0,fixed-sidebar,2,2,25.0,right_sidebar_no_categorization,6-cupcakes


### Save data to csv

In [30]:
df3.to_csv("qual_data_cleaned.csv", index=False)